In [ ]:
# Importation de la librairie pymongo
import pymongo
from datetime import datetime

# Connexion à la base de données MongoDB
client = pymongo.MongoClient(
    "mongodb://localhost:27017/"
)  # Connexion locale par défaut

# Sélection de la base de données "podcasts" (sera créée si elle n'existe pas)
db = client["podcasts"]

# Sélection de la collection "episodes" (sera créée si elle n'existe pas)
collection = db["episodes"]

# Exemple de données pour un épisode
episode = {
    "date": datetime.now(),
    "nom": "Episode 1 - Introduction",
    "duree": 3600,  # Durée en secondes
    "url": "https://example.com/episode1.mp3",
}

# Insertion de l'épisode dans la collection
resultat = collection.insert_one(episode)
print(f"ID de l'épisode inséré: {resultat.inserted_id}")

# Recherche d'un épisode par son nom
episode_trouve = collection.find_one({"nom": "Episode 1 - Introduction"})
print(f"Episode trouvé: {episode_trouve}")

ID de l'épisode inséré: 6772b289f8b3bb0f56896085
Episode trouvé: {'_id': ObjectId('6772b289f8b3bb0f56896085'), 'date': datetime.datetime(2024, 12, 30, 15, 47, 37, 882000), 'nom': 'Episode 1 - Introduction', 'duree': 3600, 'url': 'https://example.com/episode1.mp3'}
